In [79]:
import pandas as pd
import requests
#us is a module that allows conversion of US state names to abrv. and back
import us

In [68]:
r = requests.get("https://gasprices.aaa.com/state-gas-price-averages/", headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"})
results=pd.read_html(r.text)
gas_table=results[0]
gas_table

,State,Regular,Mid-Grade,Premium,Diesel
0,Alaska,$2.440,$2.546,$2.707,$2.389
1,Alabama,$1.809,$2.136,$2.466,$2.275
2,Arkansas,$1.803,$2.082,$2.374,$2.128
3,Arizona,$2.292,$2.570,$2.819,$2.501
4,California,$3.014,$3.188,$3.305,$3.299
5,Colorado,$2.333,$2.645,$2.925,$2.279
6,Connecticut,$2.079,$2.445,$2.670,$2.736
7,District of Columbia,$2.203,$2.669,$2.847,$2.804
8,Delaware,$1.993,$2.393,$2.586,$2.318
9,Florida,$2.018,$2.373,$2.663,$2.430


In [69]:
type(gas_table)

pandas.core.frame.DataFrame

In [70]:
#Make the column headers lower case to match SQL database
gas_table.columns = map(str.lower, gas_table.columns)

In [71]:
#The cost of gas is a string, need to convert
type(gas_table['regular'][1])

str

In [72]:
cols = ['regular','mid-grade','premium','diesel']

In [74]:
#drop state to lower_case
gas_table["state"] = gas_table["state"].str.lower()

In [73]:
for name in cols:
    gas_table[name] = gas_table[name].str.replace("$","")

In [75]:
gas_table.head()

,state,regular,mid-grade,premium,diesel
0,alaska,2.440,2.546,2.707,2.389
1,alabama,1.809,2.136,2.466,2.275
2,arkansas,1.803,2.082,2.374,2.128
3,arizona,2.292,2.570,2.819,2.501
4,california,3.014,3.188,3.305,3.299


In [76]:
#Change to float and round
for name in cols:
    gas_table[name] = gas_table[name].astype(float).round(2)

In [83]:
#convert states to abbr. to match craigslist

state_abbr = []

for i in range(len(gas_table['state'])):
    state = us.states.lookup(gas_table['state'][i])
    state_abbr.append(state.abbr)

In [85]:
#replace the column state with abbr.
gas_table['state'] = state_abbr

In [86]:
#drop state to lower_case
gas_table["state"] = gas_table["state"].str.lower()

In [87]:
gas_table.head()

,state,regular,mid-grade,premium,diesel
0,ak,2.44,2.55,2.71,2.39
1,al,1.81,2.14,2.47,2.28
2,ar,1.80,2.08,2.37,2.13
3,az,2.29,2.57,2.82,2.50
4,ca,3.01,3.19,3.30,3.30


In [88]:
#write the df to csv for importation into sql
gas_table.to_csv('../Resources/final_gas.csv', index=False)